# Validate network with only the models

See if the optimization model with only the ml models can generate a feasible solution

This is a relaxing problem

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Gurobi-ML-tips-modeling


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## Load licence gurobi

In [3]:
##########  LOAD LICENCE GUROBI ##########
import gurobipy as gp

# set env variable with the path of the licence
name_file_licence_gurobi = "gurobi.lic"
path_licence_gurobi = root_path + '\\' + name_file_licence_gurobi
os.environ ["GRB_LICENSE_FILE"] = path_licence_gurobi
print(os.environ["GRB_LICENSE_FILE"])

D:\github-mi-repo\Gurobi-ML-tips-modeling\gurobi.lic


In [4]:
######### LAOD CONTENT LICENCE GUROBI #########
with open(path_licence_gurobi, 'r') as f:
    content_licence = f.read()
WLSACCESSID = content_licence.split('\n')[3].split('=')[1] # load WLSACCESSID (string)
WLSSECRET = content_licence.split('\n')[4].split('=')[1] # load WLSSECRET (string)
LICENSEID = int(content_licence.split('\n')[5].split("=")[1]) # load LICENSEID (integer)

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}

## RUN

In [5]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

In [6]:
import warnings
warnings.filterwarnings('ignore')

### 1. Load configuration file optimizer and configuration instance to solve
The are principally two kind of files to config optimizer
- **configuration file to create optimizer**: there a files used to create the optimization network such as, list of sets, list of variables, upper bound and lower bound, etc. Pricipally in this files should be parameters that doesn't change too much across the time. For example, the list of variable, is a parameter that if change, the structure of the network change and the machine learning models needs to change too

- **configuration file with instance to solve**: there files that change its values every time that the optimizer solve the problem. It represents the files with the actual values of the features, and so, this values change every time that the optimizer is executed

#### 1.1. IndexTime file

In [7]:
#################### define set ####################

## paths and files names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_indextime = 'IndexTime.xlsx'
path_indextime = path_folder_config_optimization + file_indextime

# read file
indextime = pd.read_excel(path_indextime)

# set index
index_set_time = pd.Index(indextime['IndexTime'].values)
index_set_time

Index(['t0', 't1', 't2', 't3', 't4', 't5', 't6'], dtype='object')

#### 1.2 Decision variables

In [8]:
#################### define decision variables ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_allfeatures = 'AllFeatures.xlsx'
path_allfeatures = path_folder_config_optimization + file_allfeatures

# read file
config_allfeatures = pd.read_excel(path_allfeatures)

# table
config_allfeatures

,tag,feature_name,description,clasification_name,clasification,lower,upper,rate_change
0,X1,X1,Variable de entrada al proceso A. Variable Pri...,Primary,P,0.0,1000.0,100.0
1,O1,O1,Variable de entrada al proceso A. No es una va...,Observed,O,NaN,NaN,NaN
2,O2,O2,Variable de entrada al proceso A. No es una va...,Observed,O,NaN,NaN,NaN
3,O3,O3,Variable de entrada al proceso A. No es una va...,Observed,O,NaN,NaN,NaN
4,Y1,Y1,Variable target del proceso A y Variable de en...,Target,T,0.0,400.0,100.0
5,O4,O4,Variable de entrada al proceso tanque TANK1. N...,Observed,O,NaN,NaN,NaN
6,Z1,Z1,Variable de salida del tanque X y Variable de ...,Secundary,S,0.0,1000.0,100.0
7,X2,X2,Variable de entrada al proceso B. Aparece por ...,Primary,P,0.0,1000.0,100.0
8,O5,O5,Variable de entrada al proceso B. No es una va...,Observed,O,NaN,NaN,NaN
9,O6,O6,Variable de entrada al proceso B. No es una va...,Observed,O,NaN,NaN,NaN


#### 1.3 Initial Values
-  This configuration files corresponde to the description **"configuration file with instance to solve"**. This file has the initial values to start the optimizer
-  **OBS: the decision variable that are targets of machine learning models its initial value it nos defined. For all the optimization process since t=0 to t=N all the values are predicted by ml model**

In [9]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_initvalues = 'InitialValues.xlsx'
path_initvalues = path_folder_config_optimization + file_initvalues

# read file
config_initvalues = pd.read_excel(path_initvalues)

# table
config_initvalues

,tag,feature_name,init_values
0,X1,X1,50.0
1,O1,O1,50.0
2,O2,O2,50.0
3,O3,O3,50.0
4,Y1,Y1,NaN
5,O4,O4,200.0
6,Z1,Z1,120.0
7,X2,X2,5.0
8,O5,O5,5.0
9,O6,O6,5.0


#### OBS: at this part all the parameters of decision variables and observed variables were defined. Now, it is necesary define a structure of the optimization network to have the capacity to generate whatever network of this kind of problem with the posibilty to change the number of variables, its limits, capacity of tanks and also the NUMBER of process and tanks

#### 1.4 Define models to load
In this file are defined the path to ml models used in each process. Then reading the table the optimizer can go to load the ml model

**As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [10]:
#################### define initial values ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_modelsml = 'ModelsML.xlsx'
path_modelsml = path_folder_config_optimization + file_modelsml

# read file
config_modelsml = pd.read_excel(path_modelsml)

# table
config_modelsml

,process,name_process_model,path_folder,artifact_name
0,PR_A,PR_A_Y1,process_a,lr.pkl
1,PR_B,PR_B_Y2,process_b_y2,lr.pkl
2,PR_B,PR_B_Y3,process_b_y3,lr.pkl
3,PR_C,PR_C_Y2,process_c,custom


#### 1.5 Map tanks
Read a configuration table that map each tank in the process. For each tank is mapped the input flows and the output flows

Important, obviosly, the input and outputs of the tank needs to be defined in the table that maps all the features in the network

In [11]:
#################### define map input outputs each tank ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_maptanks = 'MapTanks.xlsx'
path_maptanks = path_folder_config_optimization + file_maptanks

# read file
config_maptanks = pd.read_excel(path_maptanks)

# table
config_maptanks

,tank,tag,feature_name,input_output
0,TK_1,Y1,Y1,IN
1,TK_1,O4,O4,IN
2,TK_1,Z1,Z1,OUT
3,TK_1,TL1,TL1,L
4,TK_2,Y2,Y2,IN
5,TK_2,TL2,TL2,L
6,TK_3,Y3,Y3,IN
7,TK_3,TL3,TL3,L


#### 1.6 Map process Machine learning models features and target
When each machine learning model is trained, one of the outputs of the tranining process is the artifact with the model and also a master table with the features and target of the model.

- **In this example the individual master table for each model was generated automatically and the sintetized table is generated manually by the author**

- **As you can see a process can have multiple models because the process has multiple outputs. BUT each model to develop has its own and unique name**

In [12]:
#################### define map features and targer of each model for each process ####################

# paths and file names
path_folder_config_optimization = f'config/optimization_engine/config_optimization/'
file_mapprocess_mlmodels = 'MapProcessMLmodels.xlsx'
path_mapprocess_mlmodels = path_folder_config_optimization + file_mapprocess_mlmodels

# read filemapprocess_mlmodels
config_mapprocess_mlmodels = pd.read_excel(path_mapprocess_mlmodels)

# table
config_mapprocess_mlmodels

,process,name_process_model,tag,feature_name,clasificacion
0,PR_A,PR_A_Y1,X1,X1,P
1,PR_A,PR_A_Y1,O1,O1,O
2,PR_A,PR_A_Y1,O2,O2,O
3,PR_A,PR_A_Y1,O3,O3,O
4,PR_A,PR_A_Y1,Y1,Y1,T
5,PR_B,PR_B_Y2,Z1,Z1,S
6,PR_B,PR_B_Y2,X2,X2,P
7,PR_B,PR_B_Y2,O5,O5,O
8,PR_B,PR_B_Y2,O6,O6,O
9,PR_B,PR_B_Y2,Y2,Y2,T


### 2. Load ML models
Automatically load all the models and save its into a dictionary to call when it is neccesary (as a process can have multiple models, id of the dictionary is the **name_process_model** that indicates the name for a unique model in the optimization network)
- Process A - y1
- Process B - y2
- Process C - y2 (custom model)
- Process B - y3

In [13]:
config_modelsml

,process,name_process_model,path_folder,artifact_name
0,PR_A,PR_A_Y1,process_a,lr.pkl
1,PR_B,PR_B_Y2,process_b_y2,lr.pkl
2,PR_B,PR_B_Y3,process_b_y3,lr.pkl
3,PR_C,PR_C_Y2,process_c,custom


In [14]:
# process_a_y1
path_model_process_a_y1 = 'artifacts/models/process_a/'
artifact_model_process_a_y1 = 'lr.pkl'
path_model_process_a_y1 = path_model_process_a_y1 + artifact_model_process_a_y1
model_process_a_y1 = pd.read_pickle(path_model_process_a_y1)
model_process_a_y1

Pipeline(steps=[('scaler', StandardScaler()), ('lr', LinearRegression())])

In [15]:
# process_b_y2
path_model_process_b_y2 = 'artifacts/models/process_b_y2/'
artifact_model_process_b_y2 = 'lr.pkl'
path_model_process_b_y2 = path_model_process_b_y2 + artifact_model_process_b_y2
model_process_b_y2 = pd.read_pickle(path_model_process_b_y2)
model_process_b_y2

Pipeline(steps=[('scaler', StandardScaler()), ('lr', LinearRegression())])

In [16]:
# process_b_y3
path_model_process_b_y3 = 'artifacts/models/process_b_y3/'
artifact_model_process_b_y3 = 'lr.pkl'
path_model_process_b_y3 = path_model_process_b_y3 + artifact_model_process_b_y3
model_process_b_y3 = pd.read_pickle(path_model_process_b_y3)
model_process_b_y3

Pipeline(steps=[('scaler', StandardScaler()), ('lr', LinearRegression())])

In [17]:
# process_c_y2
# CUSTOM

# parametes alpha
alpha_feature_X3 = 1/5
alpha_feature_O7 = 15

#target_predicted = alpha_feature_X3 * X_train['X3'] + alpha_feature_O7 * X_train['07']
#target_predicted

### 2. Create gurobi model

In [18]:
# # create model with licence
env = gp.Env(params=params)
model_opt = gp.Model('Example Optimization Model', env = env)

# create model without licence
# model_opt = gp.Model('Example Optimization Model')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2441807
WLS license 2441807 - registered to CMPC Celulosa S.A


### 3. Create decision variables
It is necesary to have:
- list of elements of the sets of decision var
- table with the list of decision variables to create with important considerations when the are created (for example, upper bound and lower bound)
- **create the decision var and save it into a dictionary of decision var. Use the dictionary to call the decision var when they are used to create a constraints**

#### Aditional, when the decision var is created, set the initial value

- Fix the values of period t=0 for each decision var.

- t=0 represent the actual period or initial period and it in some problems and modelations is kwown

- In addition in this notebook, the values in time t=0 are fixed for all decision variables, inclusive if the decision var have a constraint that define its values in time t = 0 (so, this kind of constraints needs to be defined since t = 1)

In [19]:
### MANUALLY - create a decision variable with the prefix var_xxx
var_X1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Y1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O4 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Z1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_X2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O5 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O6 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Y2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_Y3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_X3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_O7 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_TL1 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_TL2 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

var_TL3 = gppd.add_vars(model_opt, 
                       index_set_time, 
                       name = 'description X1',
                       ub = gp.GRB.INFINITY
                      )

### 4. Set initial values decision variables
Set the initial values t=0 for the decision variables that needs this initial values and save in t=0

In [20]:
# MANUALLY

In [21]:
# X1
init_value_X1 = 50.0
model_opt.addConstr(var_X1['t0'] == init_value_X1, name = f'Initial Value X0')

# O1
init_value_O1 = 50.0
model_opt.addConstr(var_O1['t0'] == init_value_O1, name = f'Initial Value X0')

# O2	
init_value_O2 = 50.0
model_opt.addConstr(var_O2['t0'] == init_value_O2, name = f'Initial Value X0')

# O3
init_value_O3 = 50.0
model_opt.addConstr(var_O3['t0'] == init_value_O3, name = f'Initial Value X0')

# O4
init_value_O4 = 200.0
model_opt.addConstr(var_O4['t0'] == init_value_O4, name = f'Initial Value X0')

# Z1
init_value_Z1 = 120
model_opt.addConstr(var_Z1['t0'] == init_value_Z1, name = f'Initial Value X0')

# X2
init_value_X2 = 5
model_opt.addConstr(var_X2['t0'] == init_value_X2, name = f'Initial Value X0')

# O5
init_value_O5 = 5
model_opt.addConstr(var_O5['t0'] == init_value_O5, name = f'Initial Value X0')

# O6
init_value_O6 = 5
model_opt.addConstr(var_O6['t0'] == init_value_O6, name = f'Initial Value X0')

# X3
init_value_X3 = 5
model_opt.addConstr(var_X3['t0'] == init_value_X3, name = f'Initial Value X0')

# O7
init_value_O7 = 4
model_opt.addConstr(var_O7['t0'] == init_value_O7, name = f'Initial Value X0')

# TL1
init_value_TL1 = 500
model_opt.addConstr(var_TL1['t0'] == init_value_TL1, name = f'Initial Value X0')

# TL2
init_value_TL2 = 500
model_opt.addConstr(var_TL2['t0'] == init_value_TL2, name = f'Initial Value X0')

# TL3
init_value_TL3 = 500
model_opt.addConstr(var_TL3['t0'] == init_value_TL3, name = f'Initial Value X0')

<gurobi.Constr *Awaiting Model Update*>

### 5. Lower bound and Upper bound decision variables
For example, lower bound and upper bound of tank level

---
If one decision variables doesn't need a lower bound and upper bound parameter, its value in configuration file is saved as np.NaN

Then filter the configuration table to have only the decision variables that have defined a rate_change

define in differents files one for lower bounds and another for upper bounds

#### 5.1 Lower bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [22]:
# MANUALLY

In [23]:
# var_X1
var_X1_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_X1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_X1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Y1
var_Y1_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Y1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Y1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Z1
var_Z1_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Z1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Z1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_X2
var_X2_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_X2,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_X2_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Y2
var_Y2_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Y2,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Y2_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_Y3
var_Y3_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_Y3,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_Y3_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_X3
var_X3_lower_bound = 0
gppd.add_constrs(model_opt,
                 var_X3,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_X3_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_TL1
var_TL1_lower_bound = 100
gppd.add_constrs(model_opt,
                 var_TL1,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_TL1_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_TL2
var_TL2_lower_bound = 100
gppd.add_constrs(model_opt,
                 var_TL2,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_TL2_lower_bound,  # lower bound
                 name = f'Lower bound X0')

# var_TL3
var_TL3_lower_bound = 100
gppd.add_constrs(model_opt,
                 var_TL3,  # decision var
                 gp.GRB.GREATER_EQUAL,
                 var_TL3_lower_bound,  # lower bound
                 name = f'Lower bound X0')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: Lower bound X0, dtype: object

#### 5.2 upper bound
Since config decision var create a subtable with only the features that have defined its upper bound and then create the constraints

In [24]:
##### MANUALLY

In [25]:
# var_X1
var_X1_upper_bound = 1000
gppd.add_constrs(model_opt,
                 var_X1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_X1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Y1
var_Y1_upper_bound = 400
gppd.add_constrs(model_opt,
                 var_Y1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Y1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Z1
var_Z1_upper_bound = 1000
gppd.add_constrs(model_opt,
                 var_Z1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Z1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_X2
var_X2_upper_bound = 1000
gppd.add_constrs(model_opt,
                 var_X2,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_X2_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Y2
var_Y2_upper_bound = 500
gppd.add_constrs(model_opt,
                 var_Y2,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Y2_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_Y3
var_Y3_upper_bound = 450
gppd.add_constrs(model_opt,
                 var_Y3,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_Y3_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_X3
var_X3_upper_bound = 1500
gppd.add_constrs(model_opt,
                 var_X3,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_X3_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_TL1
var_TL1_upper_bound = 20000
gppd.add_constrs(model_opt,
                 var_TL1,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_TL1_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_TL2
var_TL2_upper_bound = 20000
gppd.add_constrs(model_opt,
                 var_TL2,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_TL2_upper_bound,  # upper bound
                 name = f'Upper bound X0')

# var_TL3
var_TL3_upper_bound = 20000
gppd.add_constrs(model_opt,
                 var_TL3,  # decision var
                 gp.GRB.LESS_EQUAL,
                 var_TL3_upper_bound,  # upper bound
                 name = f'Upper bound X0')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: Upper bound X0, dtype: object

### 8. Set values of observed variables
The observed variables are not decision variables, but with the idea to automatize this codes they are defined as decision variables and then a constraint is added to set its values to a unique values

Doing this is a little automatice the creation of the constraints and open the posibility to transform the observed variables into a decision variables and connect other models/tanks/process to this observed variables with a less modification in the codes

In [26]:
##### MANUALLY

In [27]:
# var_O1

observed_value_O1 = 50.0
gppd.add_constrs(model_opt,
                 var_O1,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O1,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

In [28]:
# var_O2

observed_value_O2 = 50.0
gppd.add_constrs(model_opt,
                 var_O2,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O2,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

In [29]:
# var_O3

observed_value_O3 = 50.0
gppd.add_constrs(model_opt,
                 var_O3,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O3,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

In [30]:
# var_O4

observed_value_O4 = 200.0
gppd.add_constrs(model_opt,
                 var_O4,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O4,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

In [31]:
# var_O5

observed_value_O5 = 5.0
gppd.add_constrs(model_opt,
                 var_O5,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O5,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

In [32]:
# var_O6

observed_value_O6 = 5.0
gppd.add_constrs(model_opt,
                 var_O6,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O6,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

In [33]:
# var_O7

observed_value_O7 = 4.0
gppd.add_constrs(model_opt,
                 var_O7,  # decision var
                 gp.GRB.EQUAL,
                 observed_value_O7,  # value observed variable
                 name = f'set value to observed variable')

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: set value to observed variable, dtype: object

### 8. Load a Machine Learning Model as constraints that represent the relations in the process
Add a constraint defined as a machine learning model that represent the relation between X1, X2 with the output of the process (Y1)

In [34]:
############# Connect model process_a_y1

# Generate instance of machine learning model
instance = pd.DataFrame([var_X1, var_O1, var_O2, var_O3]).T
instance.columns = ["X1", "O1", "O2", "O3"]

# Add machine learning model as constraint

# pred_constr_ml_process_a_y1 = add_predictor_constr(gp_model = model_opt, 
#                                                    predictor = model_process_a_y1, 
#                                                    input_vars = instance, # instance pandas gurobi
#                                                    output_vars = var_Y1, # target
#                                                    name = f'model_predict Y1'
#                                                   )

# pred_constr_ml_process_a_y1.print_stats()

In [35]:
############# Connect model model_process_b_y2


# Generate instance of machine learning model
instance = pd.DataFrame([var_Z1, var_X2, var_O5, var_O6]).T
instance.columns = ["Z1", "X2", "O5", "O6"]

# Add machine learning model as constraint

# pred_constr_ml_model_process_b_y2 = add_predictor_constr(gp_model = model_opt, 
#                                                    predictor = model_process_b_y2, 
#                                                    input_vars = instance, # instance pandas gurobi
#                                                    output_vars = var_Y2, # target
#                                                    name = f'model_predict Y2'
#                                                   )

# pred_constr_ml_model_process_b_y2.print_stats()

In [36]:
############# Connect model model_process_b_y3


# Generate instance of machine learning model
instance = pd.DataFrame([var_Z1, var_X2]).T
instance.columns = ["Z1", "X2"]

# Add machine learning model as constraint

pred_constr_ml_model_process_b_y3 = add_predictor_constr(gp_model = model_opt, 
                                                   predictor = model_process_b_y3, 
                                                   input_vars = instance, # instance pandas gurobi
                                                   output_vars = var_Y3, # target
                                                   name = f'model_predict Y3'
                                                  )

pred_constr_ml_model_process_b_y3.print_stats()

Model for model_predict Y3:
14 variables
21 constraints
Input has shape (7, 2)
Output has shape (7, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
std_scaler            (7, 2)           14           14            0            0

lin_reg               (7, 1)            0            7            0            0

--------------------------------------------------------------------------------


In [53]:
var_X2

t0    <gurobi.Var description X1[t0] (value 5.0)>
t1    <gurobi.Var description X1[t1] (value 0.0)>
t2    <gurobi.Var description X1[t2] (value 0.0)>
t3    <gurobi.Var description X1[t3] (value 0.0)>
t4    <gurobi.Var description X1[t4] (value 0.0)>
t5    <gurobi.Var description X1[t5] (value 0.0)>
t6    <gurobi.Var description X1[t6] (value 0.0)>
Name: description X1, dtype: object

### 8. Define a custom function as constraints that represent the relations in the process C

**Important: this constraint and the objective functions are more hardoded that the rest of constraints**

In [37]:
# define parameters of the constraint
alpha_feature_1 = 1/5
alpha_feature_2 = 15

In [38]:
# define function as constraint
gppd.add_constrs(model_opt, 
                 (alpha_feature_1 * var_X3 + alpha_feature_2 * var_O7), 
                 gp.GRB.EQUAL, 
                 var_Y2, 
                 name = 'function as constraint output process C'
                )

t0    <gurobi.Constr *Awaiting Model Update*>
t1    <gurobi.Constr *Awaiting Model Update*>
t2    <gurobi.Constr *Awaiting Model Update*>
t3    <gurobi.Constr *Awaiting Model Update*>
t4    <gurobi.Constr *Awaiting Model Update*>
t5    <gurobi.Constr *Awaiting Model Update*>
t6    <gurobi.Constr *Awaiting Model Update*>
Name: function as constraint output process C, dtype: object

In [39]:
model_opt.update()

In [40]:
model_opt

<gurobi.Model Continuous instance Example Optimization Model: 231 constrs, 133 vars, Parameter changes: LicenseID=2441807>

### 9. Define objective optimization
Objetive that no generate infeasibility

In [41]:
var_Y3.sum()

<gurobi.LinExpr: description X1[t0] + description X1[t1] + description X1[t2] + description X1[t3] + description X1[t4] + description X1[t5] + description X1[t6]>

In [42]:
var_Y2.sum()

<gurobi.LinExpr: description X1[t0] + description X1[t1] + description X1[t2] + description X1[t3] + description X1[t4] + description X1[t5] + description X1[t6]>

In [43]:
# optization
model_opt.setObjective(var_Y3.sum() + var_Y2.sum(),
                       gp.GRB.MAXIMIZE)

## ----> return the mdoel_opt that has defined for this instance can get the solution <----

In [44]:
model_opt

<gurobi.Model Continuous instance Example Optimization Model: 231 constrs, 133 vars, Parameter changes: LicenseID=2441807>

### 10. Optimize and get optimal values

In [45]:
# solve
model_opt.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19043.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

WLS license 2441807 - registered to CMPC Celulosa S.A
Optimize a model with 231 rows, 133 columns and 273 nonzeros
Model fingerprint: 0x8850a074
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+04]
Presolve removed 231 rows and 133 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1605000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  5.160500000e+03


In [46]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
model_opt.Status

2

In [47]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set_time)

######################## save optimal values - features of models (only the features) ########################
solution["var_X1"] = var_X1.gppd.X
solution["var_O1"] = var_O1.gppd.X
solution["var_O2"] = var_O2.gppd.X
solution["var_O3"] = var_O3.gppd.X
solution["var_Y1"] = var_Y1.gppd.X
solution["var_O4"] = var_O4.gppd.X
solution["var_Z1"] = var_Z1.gppd.X
solution["var_X2"] = var_X2.gppd.X
solution["var_O5"] = var_O5.gppd.X
solution["var_O6"] = var_O6.gppd.X
solution["var_Y2"] = var_Y2.gppd.X
solution["var_Y3"] = var_Y3.gppd.X
solution["var_X3"] = var_X3.gppd.X
solution["var_O7"] = var_O7.gppd.X
solution["var_TL1"] = var_TL1.gppd.X
solution["var_TL2"] = var_TL2.gppd.X
solution["var_TL3"] = var_TL3.gppd.X

######################## # get value objetive function ########################
opt_objetive_function = model_opt.ObjVal

In [48]:
# show value objetive function
opt_objetive_function

5160.5

In [49]:
solution

,var_X1,var_O1,var_O2,var_O3,var_Y1,var_O4,var_Z1,var_X2,var_O5,var_O6,var_Y2,var_Y3,var_X3,var_O7,var_TL1,var_TL2,var_TL3
t0,50.0,50.0,50.0,50.0,0.0,200.0,120.0,5.0,5.0,5.0,61.0,239.5,5.0,4.0,500.0,500.0,500.0
t1,0.0,50.0,50.0,50.0,0.0,200.0,225.0,0.0,5.0,5.0,360.0,450.0,1500.0,4.0,100.0,100.0,100.0
t2,0.0,50.0,50.0,50.0,0.0,200.0,225.0,0.0,5.0,5.0,360.0,450.0,1500.0,4.0,100.0,100.0,100.0
t3,0.0,50.0,50.0,50.0,0.0,200.0,225.0,0.0,5.0,5.0,360.0,450.0,1500.0,4.0,100.0,100.0,100.0
t4,0.0,50.0,50.0,50.0,0.0,200.0,225.0,0.0,5.0,5.0,360.0,450.0,1500.0,4.0,100.0,100.0,100.0
t5,0.0,50.0,50.0,50.0,0.0,200.0,225.0,0.0,5.0,5.0,360.0,450.0,1500.0,4.0,100.0,100.0,100.0
t6,0.0,50.0,50.0,50.0,0.0,200.0,225.0,0.0,5.0,5.0,360.0,450.0,1500.0,4.0,100.0,100.0,100.0


In [50]:
# var_y2 = 100
# var_x3 = 200
# var_o7 = 4
200 * 1/5 + 4*15 # output tag y2

100.0

In [51]:
# tabla de limites operacionales
config_allfeatures.dropna()

,tag,feature_name,description,clasification_name,clasification,lower,upper,rate_change
0,X1,X1,Variable de entrada al proceso A. Variable Pri...,Primary,P,0.0,1000.0,100.0
4,Y1,Y1,Variable target del proceso A y Variable de en...,Target,T,0.0,400.0,100.0
6,Z1,Z1,Variable de salida del tanque X y Variable de ...,Secundary,S,0.0,1000.0,100.0
7,X2,X2,Variable de entrada al proceso B. Aparece por ...,Primary,P,0.0,1000.0,100.0
10,Y2,Y2,Variable target del proceso B y proceso C (y v...,Target,T,0.0,500.0,100.0
11,Y3,Y3,Variable target del proceso B que finaliza el ...,Target,T,0.0,450.0,100.0
12,X3,X3,Variable de entrada al proceso C. Aparece por ...,Primary,P,0.0,1500.0,100.0
14,TL1,TL1,Tank level - tank 1,Tank level,L,100.0,20000.0,300.0
15,TL2,TL2,Tank level - tank 2,Tank level,L,100.0,20000.0,300.0
16,TL3,TL3,Tank level - tank 3,Tank level,L,100.0,20000.0,300.0
